## Scala a functional programming approach 2: Data Types


#### Abstract
This document is a efford to introduce the strengths and benefits of functional programming in scala.

We do not claim intelectual property of all the material presented.
We specifically refer to the original resources whenever is needed.

The presentation path of the consepts is still under consideration and may be changed in future reviews.

### Outline
- Polymorphism
- Parametric polymorphism
- Pattern matching
- Basic immutable collections
- Basic ADTs 



#### [Scala Immutable List](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html)



In [15]:
// Immutable List data type
import scala.collection.immutable.List


// Constructing lists
val numbers = List(1,2,3)
val moreNumbers: List[Int] = 4 :: 5 :: Nil

// Operations
val head = numbers.head
val tail = numbers.tail
val init = numbers.init
val last = numbers.last
val reverse = numbers.reverse


//Concatenation
numbers ++ moreNumbers

//Prepend
0 :: numbers // = numbers.::(0)
0 +: numbers

//Append 
moreNumbers :+ 6


import scala.collection.immutable.List
numbers: List[Int] = List(1, 2, 3)
moreNumbers: List[Int] = List(4, 5)
head: Int = 1
tail: List[Int] = List(2, 3)
init: List[Int] = List(1, 2)
last: Int = 3
reverse: List[Int] = List(3, 2, 1)
res14_8: List[Int] = List(1, 2, 3, 4, 5)
res14_9: List[Int] = List(0, 1, 2, 3)
res14_10: List[Int] = List(0, 1, 2, 3)
res14_11: List[Int] = List(4, 5, 6)

#### Example:  Recreating the immutable list

In [3]:
// Recreating immutable list

sealed trait Lst[+A]
case object Nil extends Lst[Nothing]
case class Cons[A](head: A, tail: Lst[A]) extends Lst[A]


val empty: Lst[Int] = Nil
val numbers: Lst[Int] = Cons(1, Cons(2, Cons(3, Nil)))


object Lst {

  def apply[A](ss: A*): Lst[A] =      
      if(ss.isEmpty) Nil
      else Cons(ss.head, apply(ss.tail: _*))
}

Lst(1,2,3) //Now we can write

defined trait Lst
defined object Nil
defined class Cons
empty: $user.Lst[Int] = Nil
numbers: $user.Lst[Int] = Cons(1,Cons(2,Cons(3,Nil)))
defined object Lst
res2_6: $user.Lst[Int] = Cons(1,Cons(2,Cons(3,Nil)))

#### Pattern matching

In [17]:
def funnyMatch(l: Lst[String]): String =  l match {
  case (Cons(x, Cons("2", Cons(y, _)))) => x + y
  case Nil => "Nil"
  case Cons("1", _) => "Starting with 1"
  case _ => sys.error("Oops!!!")
}

funnyMatch(Lst())

funnyMatch(Lst("test ", "2", "foo"))

funnyMatch(Lst("1", "2"))

// funnyMatch(Lst("2", "3")) throws Opps!

// funnyMatch(Lst(1,2)) type mismatch


defined function funnyMatch
res16_1: String = "Nil"
res16_2: String = "test foo"
res16_3: String = "Starting with 1"

#### Implementing list operations

In [4]:
object Lst {

    def apply[A](ss: A*): Lst[A] =      
      if(ss.isEmpty) Nil
      else Cons(ss.head, apply(ss.tail: _*))
      
    def
      
}
Lst(1,2,3)

defined object Lst
res3_1: Lst[Int] = Cons(1,Cons(2,Cons(3,Nil)))

### Resources

- [Functional programming in scala](https://www.amazon.com/Functional-Programming-Scala-Paul Chiusano/dp/1617290653/ref=sr_1_1?ie=UTF8&qid=1504870248&sr=8-1&keywords=functional+programming+in+scala)

 _Fotios Paschos, `@fpaschos` Sep, 2017_